In [1]:
import os
import sys
import time
import pprint
from collections import Counter, defaultdict
sys.path.append('/home/psavine/source/viper')
sys.path.append("../")


import numpy as np
import importlib
import matplotlib.pyplot as plt

import spec.hvac as thvac
import src.viper as viper

import src.geom as geom
import lib.geo as geo
from src.geomType import GeomType
from scipy import spatial
import src.ui.visual as viz3


from trimesh.path.path import Path
from src.ui import visual
from itertools import chain
from src.ui.visual import meshcat
import shapely.geometry as sg
import trimesh
import pandas as pd
import matplotlib
import math
import json
import spec.scenario as scene
import src.geombase as geombase

import src.formats.skansk as sks
import src.propogate as gp
from src.structs import Node, Edge
import src.structs.node_utils as gutils
import src.formats.revit as revit


RL = importlib.reload
# RL(visual)
VIZ = viz3.VIZ

_dir = '/home/psavine/source/viper/data/sample/'


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [2]:
jsond = scene.load_from_json('/home/psavine/source/viper/data/sample/test_sk2.json')

RL(thvac)
train_mesh = thvac.TestTrain.preprocess_(jsond)
print(len(train_mesh))
categories = {x.category for i, x in enumerate(train_mesh)}
categories = {x:i for i, x in enumerate(categories)}
train_ids = {x.id:x for x in train_mesh}
train_ixs = {x.id:i for i, x in enumerate(train_mesh)}
vs2 = viper.IndexSystem()
# ixrs = vs2(train_mesh)

4439


In [3]:
def as_line(this, k='point'):
    pts = this.children_of_type(k)
    if len(pts) == 2:
        p1, p2 = pts
        return geo.Line(geo.Point(p1.geom), geo.Point(p2.geom))

    
def as_linexf(this, trs=np.array([0.,0.,0.]), scale=1.):
    pts = this.children_of_type('point')
    if len(pts) == 2:
        xf = this.children_of_type('transform')
        if len(xf) == 1:
            try:
                p1, p2 = pts
                p11 = (xf[0].apply(p1.geom) *  scale) + trs
                p12 = (xf[0].apply(p2.geom) *  scale) + trs
                return geo.Line(geo.Point(p11), geo.Point(p12))
            except Exception as e:
                return None


def dxf_line_to_line(dxline, bsxf=np.array([0.,0.,0.])):
    p1, p2 = [np.array(x) - bsxf for x in [dxline.start, dxline.end]] 
    return geo.Line(geo.Point(p1), geo.Point(p1))


def pull_points(acis_blocks):
    for i, acis_ents in enumerate(acis_blocks):
        pts = list(filter(lambda x: isinstance(x, sab.ACEllipse), acis_ents))
        if len(pts) == 2:
            p1, p2 = pts
            viz3.viz_line(p1.geom, p2.geom, handle='cad/{}'.format(i))


def viz_pts(pts, handle, clear=True, **kwargs):
    if clear is True:
        VIZ[handle].delete()
    for i, p in enumerate(pts):
        viz3.viz_point(p, handle=handle+'/{}'.format(i), **kwargs)


def viz_nodes(ndict):
    if isinstance(ndict, dict):
        nlist = list(ndict.values())
    else:
        nlist = ndict
    viz3.viz_nodes(nlist, 
                   handle='nodes', 
                   thickness=0.08, 
                   opacity=0.6, 
                   clear=True)

def viz_head():
    ddd = [viz3.viz_point(np.array(x.insert) - bsxf,  
                          handle='cadhead/{}'.format(i), 
                          radius=3, 
                          opacity=0.5) 
           for i, x in enumerate(sprks)]
    

def viz_sprink_to_edge(sbests, sprinks):
    VIZ['headlink'].delete()
    for i, (spr_ix, ngh) in enumerate(sbests.items()):
        if ngh is None:
            continue
        g1, g2 = ngh.geom
        pt3d = geo.Point(sprinks[spr_ix])
        cp = geo.Point(g1).midpoint_to(geo.Point(g2))
        viz3.viz(geo.Line(pt3d, cp), handle='headlink/{}'.format(i))

### IMPORT DXF 

In [4]:
import dxfgrabber
dxf = dxfgrabber.readfile(_dir + 'WGFPR07m.dxf')
bodies = list(filter(lambda x: isinstance(x, dxfgrabber.dxfentities.Body), dxf.entities))
lines = list(filter(lambda x: isinstance(x, dxfgrabber.dxfentities.Line), dxf.entities))
inserts = list(filter(lambda x: isinstance(x, dxfgrabber.dxfentities.Insert), dxf.entities))

In [5]:
import src.formats.sat as sab
RL(sab)
reader = sab.SABReader()
acis_ents = [reader.read_single(x.acis) for x in bodies]

In [6]:
CAD_SCALE = 1 /  25.39999999999999858 
ANGLE = 346.9658914 
CAD_TRANSLATE = -1* np.array(list(as_linexf(acis_ents[0], scale=CAD_SCALE).points[0].coordinates))

# Create Final Lines and Visualize

In [7]:
sprinklers = [x for x in inserts if x.name == '3DM_UPR']
print('num sprinklers' , len(sprinklers))
# print(mv)
print(CAD_TRANSLATE)
sample = np.array(sprinklers[2].insert)
print(sample)
sfn = lambda x: CAD_TRANSLATE  + (np.asarray(x.insert) * CAD_SCALE)

sprinklers2 = np.array(list(map(sfn, sprinklers)))
sprinklers2[:, -1] = 0
print(sprinklers2[2])

viz_pts(sprinklers2, 'cadhead', clear=True, radius=0.2, opacity=0.5)

num sprinklers 1403
[-15.4164401  312.02305601 -44.87684516]
[  463.1242295  -8199.00927198  1126.13126295]
[  2.81679729 -10.77258462   0.        ]


In [8]:
from scipy.spatial.distance import cdist 
import src.formats.skansk as sks
RL(sks)

def edge_with_id(root_node, eid):
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, True):
            if e.id == eid:
                return e

def node_with_id(nd, eid):
    for n in nd.__iter__(fwd=True, bkwd=True):
        if n.id == eid:
            return n

        
def apply_fn(root_node, fn, edges=True):
    seen = set()
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, edges):
            if e.id not in seen:
                seen.add(e.id)
                res = fn(e)
                if res is not None:
                    yield e

                    
def apply_node_edge_fn(root_node, fn):
    seen = set()
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, True):
            if e.id not in seen:
                seen.add(e.id)
                res = fn(n, e)
                if res is not None:
                    yield res           

                    
def wrap_yield(fn):
    def run_x(x):
        res = fn(x)
        if res is True:
            return x
        else:
            return None
    return run_x


def viz_edge_attr(root_node, k, **kwargs):
    for n in root_node.__iter__(True, True):
        for e in n.neighbors(True, True, True):
            if  e.get(k) is None:
                continue
            g1, g2 = e.geom
            viz3.viz_line(g1, g2, handle=viz3.as_path(k,  e.id), **kwargs)


def viz_node_attr(root_node, k, v=None, clear=True, **kwargs):
    if clear is True:
        VIZ[k].delete()
    for n in root_node.__iter__(True, True):
        if n.get(k) is None:
            continue
        if v is not None and n.get(k) != v:
            continue
        viz3.viz_point(n.geom, handle=viz3.as_path(k,  n.id), **kwargs)
            
            
            
def as_linecyl(this, trs=np.array([0.,0.,0.]), scale=1.):
    line = (this.line.numpy * scale) + trs
    rads = this.radius * scale
    this.line = geo.Line(geo.Point(line[0]), geo.Point(line[1]))
    this.radius = rads
    return this


def fix_triangles(root_node_, result):
    for lres in result:
        tris = [edge_with_id(root_node_, x) for x in lres]
        tris = [x for x in tris if x is not None]
        if len(tris) == 3:
            sks.resolve_triangle(tris)
        
        
def xfrom(node, scale=1., translate=None, **kwargs):
    nd = np.array(list(node.geom)) * scale
    if translate is not None:
        nd += translate
    node.geom = sks.tuplify(nd)
    return node

In [9]:
from lib.geo import Point, Line
RL(sab)
reader = sab.SABReader()
acis_ents2 = list(filter(lambda x: x.valid is True,
                       map(sab.SACCylinder,
                           acis_ents)))
                           #map(lambda x: reader.read_single(x.acis),
                           #    bodies))))
print(len(acis_ents2))

pipes_cyl = list(
    filter(lambda x: x.line.length > x.radius,
    filter(lambda x: x and x.valid is True, 
        map(lambda x: as_linecyl(x,scale=CAD_SCALE, trs=CAD_TRANSLATE) 
                           ,acis_ents2))))
print(len(pipes_cyl))

14427
8208


In [10]:

RL(gp)
RL(gutils)
RL(sks)

import src.ui.visual as V

# V.viz_order(rn1, handle='build')
# RL(viz3)
"""
--------------------------------------------------
SAVE THESE SETTINGS. 

FOR FUCKS SAKE DO NOT DELETE !!!!!!!!!!!!!!!

FILTER = 0.3
CONNECT = 0.4
--------------------------------------------------
"""
CAD_ANGLE = np.radians(360 - ANGLE)
REVIT_SCALE = 1 / 0.4799

RL(V)
def save_instr(data):
    _COMMS_PATH = '/home/psavine/source/viper/data/out/commands/comms.json'
    
    print('creating insturcionts', len(data))
    with open(_COMMS_PATH, 'w' ) as f:
        json.dump(data, f)
    return data


def build_roots():
    RL(sks)
    FILTER = 0.3
    CONNECT = 0.4
    
     # Build the Graph
    pipes_long = [x for x in pipes_cyl if x.line.length > FILTER]
    pairs2, tree, pts_ind = sks.connect_by_pairs(pipes_long, factor=CONNECT)
    node_dict2 = sks.make_node_dict(pairs2, pts_ind, pipes_long)
    root_nodes = sks.connected_components(list(node_dict2.values()), min_size=2)
    return root_nodes

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [11]:
def viz_kif(root_, k, i, **kwargs):
    # print(k, kwargs.get(k, None))
    if kwargs.get(k, None) is True:
        viz3.viz_iter(root_, handle='grp/{}/{}'.format(i, k), 
                      clear=True, opacity=0.3)
    else:
        viz3.delete(k)

shows = dict(heads=True, edge=True, 
             before=True, buildo=True, tris=True, move=True)

def xprocess(_Nodes, ix=0, v=True, **kwargs):
    # Find Logical Root
    RL(sks)
    RL(gp)
    base_root_node = _Nodes[ix]
    SR = gp.SpatialRoot()
    SR(base_root_node)
    Root = node_with_id(base_root_node, SR.best)

    Mover = gp.Rotator(CAD_ANGLE)
    Elbower = sks.DetectElbows()
    Triangler = sks.DetectTriangles(build=True)
    
    # Direct the edges 
    Root = gp.EdgeDirector()(Root)
    Root = gp.BuildOrder()(Root)

    tree = sks.resolve_heads(Root, sprinklers2)
    # viz_kif(base_root_node, 'heads', ix, **kwargs)
    
    Root = Mover(Root)
    # viz_kif(base_root_node, 'move', ix, **kwargs)
    # V.viz_iter(Root, handle='grp/{}/pre'.format(ix), clear=True, opacity=0.4)
    
    Root = gp.EdgeDirector()(Root)
    Root = gp.BuildOrder()(Root)
    Root = Triangler(Root)
    Root = gp.EdgeDirector()(Root)
    
    # V.viz_iter(Root, handle='grp/{}/x1'.format(ix), clear=True, opacity=0.4)
    Root = gp.FunctionQ(sks.add_ups)(Root, h=0.1)
    Root = gp.GraphTrim()(Root, tol=0.01)
    # V.viz_iter(Root, handle='grp/{}/tri'.format(ix), clear=True, opacity=0.4)
    
    Root = gp.EdgeDirector()(Root)
    Root = gp.FunctionQ(sks.remove_short)(Root, lim=0.25)
    
    # V.viz_iter(Root, handle='grp/{}/x3'.format(ix), clear=True, opacity=0.4)
    
    FirstEdges = Root.neighbors(edges=True)
    Elbower(FirstEdges)
    Root = gp.EdgeDirector()(Root)
    Root = gp.Scale(REVIT_SCALE)(Root)
    
    Root = sks.LabelConnector()(Root)
    FirstEdges = Root.neighbors(edges=True)

    Root = gp.FunctionQ(sks.resolve_elbow_edges)(Root)
    gp.FunctionQ(sks.align_tap, edges=True)(FirstEdges)
    
    if v is True:
        V.viz_iter(Root, handle='grp/{}/final'.format(ix), clear=True, opacity=0.4)

    else:
        V.delete('grp/{}/final'.format(ix))
    return Root # , tree

## Building Graphs

In [12]:
RL(revit)
RL(sks)
RL(geombase)
import src.propogate.threedprop
import src.propogate.geometric
RL(src.propogate.geometric)
RL(src.propogate.threedprop)
RL(gp)

[V.delete(x) for x in ['tee1', 'edge', 'tee2', 'tee3', 'long', 'short']]
root_nodes = build_roots()
rn0  = xprocess(root_nodes,  ix=0, v=True)

#instr = sks.MakeInstructionsNodeBased()
#res = instr(rn0, shrink=-0.05, use_radius=True)
#_ =save_instr(res)

# viz_node_attr(rn0, 'tee_mid', clear=True, radius=0.05)
# viz_node_attr(rn0, 'tee_index', clear=True, radius=0.03)

SpatialRoot seen nodes :  330
EdgeDirector seen nodes :  330
BuildOrder seen nodes :  330
KDTreeIndex seen nodes :  330
Rotator seen nodes :  239
EdgeDirector seen nodes :  239
BuildOrder seen nodes :  239
DetectTriangles seen nodes :  229
EdgeDirector seen nodes :  211
FunctionQ seen nodes :  319
GraphTrim seen nodes :  319
EdgeDirector seen nodes :  297
FunctionQ seen nodes :  290
DetectElbows seen nodes :  289
EdgeDirector seen nodes :  290
Scale seen nodes :  290
LabelConnector seen nodes :  290
FunctionQ seen nodes :  274
FunctionQ seen nodes :  273


In [13]:
RL(revit)
RL(sks)
# instr = sks.MakeInstructionsV3()
# res = instr(rn0, shrink=-0.05, use_radius=True)

import networkx as nx
import src.misc.utils as utils
RL(revit)

instr = sks.MakeInstructionsV3()
res = instr(rn0, shrink=-0.05, use_radius=True)
rn0.write(root=True)

G = utils.nodes_to_nx(rn0)

nxpath = '/home/psavine/source/viper/data/out/commands/graphtest.pkl'
nx.write_gpickle(G, nxpath)


MakeInstructionsV3 seen nodes :  274


## Testing Comms Server Functions Local

In [80]:
import src.formats.revit_base as rvb
RL(rvb)
RL(revit)

# 
rn0 = revit.ITee.remove(rn0)
bld = revit.ITee(rn0) # , strategy=revit.ITee.FromGeometryPlacement)
a1 = bld.next_action()
# bld.on_success(a1)
bld.on_fail(a1)
a2 = bld.next_action()
a3 = bld.next_action()
a4 = bld.next_action()
[print(x) for x in [a1, a2, a3, a4]]

rn0.tmps

[<class 'src.formats.revit.ITee.FromGeometryPlacement'>]
[5, 4459043915, 'Tee - Generic', 526.848367108863, -51.715957739736886, 1.6515011751184112, -0.5773502693683984, 0.5773502691002396, -0.5773502691002395, 2.0943951021250364, 0.9999916691351617, 4.6446102012275806e-10, -0.0, -0.0, 1.5667144457565891, 0.03740157480314961]
[1, 4075015015, 4075015015, 526.848367108863, -51.715957739736886, 1.6515011751184112, 526.8483671045805, -42.49549076014023, 1.6138641167567014, 0.07480314960629922]
[1, 5408843893, 5408843893, 526.848367108863, -51.715957739736886, 1.6515011751184112, 526.848367108863, -51.715957739736886, 1.859877920273652, 0.08333333333333333]
None


{'pt_index': 3511,
 'order': 0,
 'has_head': False,
 'head_tee': True,
 '$create': 'tee',
 '__creator': None,
 'built': False,
 'failed': True,
 'root': True}

In [19]:
nd = gutils.node_with_id(rn0, 5566475637)
nd.tmps, 

({'pt_index': 3413,
  'order': 33,
  'has_head': False,
  'head_tee': True,
  'is_elbow': True,
  '$create': 'tee'},
 1)

In [20]:
nd.successors(ix=0, edges=True).tmps

{'remove_head': True, 'tap_edge': True, '$create': 'pipe'}

## Testing Comms Server interactrively 

In [66]:
import src.comms.smartbuild as sb
import src.comms.zserver as zz
_cntrl_url = 'tcp://10.0.1.37:5556'

In [84]:
n3 = Node((0,0 ,0))
n4 = Node((10, 0, 0))
edge = n3.connect_to(n4)

xxcmd = revit.Command.create(revit.Cmds.Pipe, revit.PipeCmd.Points, edge)

In [71]:
RL(sb); RL(zz)
CNC = zz.CommandController(_cntrl_url)
reset = CNC.reset('simple')
# CNC.state()


cmd2 = [[revit.Cmds.Elbow.value, 4437351, 7312675]]
print(cmd)
CNC.update(cmd2)

print(cmd2)
CNC.update(cmd)
print(CNC.state())

Instruction updater started:
[[1, 4008875, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.08333333333333333]]
[[4, 4437351, 7312675]]
{'todo': [[4, 4437351, 7312675], [1, 4008875, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.08333333333333333]], 'failed': []}


In [91]:
e = gutils.edge_with_id(rn0, 7676757)
nd = e.source
s = nd.successors(ix=0)
t = nd.predecessors(ix=0)
cmd = revit.Command.create(revit.Cmds.FamilyOnPoint, s, nd, t, family='Elbow - Generic')

CNC.reset('simple')
CNC.update(cmd + [[11, 7676757, 111080], [11, 5362885, 111080] ] + xxcmd)


[[12, 'Elbow - Generic', 111080, 2, 328.0044764145159, 0.09002974902569827, 0.9371293045024331, 0.6551768185627052, 0.37990118003603984, 0.6530072203469702, 2.4132662299509335, 0.9973511505314635, -0.0028758585682094992, -0.001635091336944163, 1.0468430755462774, 1.6435222271366303, 0.03740157480314961]]


['SUCCESS']

In [92]:
e2 = gutils.edge_with_id(rn0, 7676757)
n2 = e.source
s2 = nd.successors(ix=0)
t2 = nd.predecessors(ix=0)
CNC.reset('simple')
CNC.update(cmd + [[11, 7676757, 111080], [11, 5362885, 111080] ] + xxcmd)

['SUCCESS']

In [124]:
RL(revit)
from enum import Enum
isinstance(revit.Cmds.Pipe, Enum)

True

MakeInstructionsV3 seen nodes :  274


In [87]:
G.nodes[(526.848367108863, -51.715957739736886, 1.6515011751184112)]

{'pt_index': 3511,
 'order': 0,
 'has_head': False,
 'head_tee': True,
 '$create': 'tee',
 'root': True}

In [234]:
RL(sks)
[V.delete(x) for x in ['tee1', 'edge', 'tee2', 'tee3', 'long', 'short']]
all_res = []
root_nodes = build_roots()
for i in range(len(root_nodes)):
    rnX  = xprocess(root_nodes,  ix=i, v=True)
    all_res += sks.MakeInstructions()(rnX, shrink=-0.05, use_radius=True)
_ = save_instr(all_res)

SpatialRoot seen nodes :  330
EdgeDirector seen nodes :  330
BuildOrder seen nodes :  330
KDTreeIndex seen nodes :  330
Rotator seen nodes :  239
EdgeDirector seen nodes :  239
BuildOrder seen nodes :  239
DetectTriangles seen nodes :  229
EdgeDirector seen nodes :  211
FunctionQ seen nodes :  319
GraphTrim seen nodes :  319
EdgeDirector seen nodes :  297
FunctionQ seen nodes :  290
DetectElbows seen nodes :  289
EdgeDirector seen nodes :  290
Scale seen nodes :  290
LabelConnector seen nodes :  290
adjusting  7031568
adjusting  6300107
adjusting  7677216
adjusting  9201100
adjusting  7899817
adjusting  2507759
FunctionQ seen nodes :  289
MakeInstructions seen nodes :  290
SpatialRoot seen nodes :  452
EdgeDirector seen nodes :  452
BuildOrder seen nodes :  452
KDTreeIndex seen nodes :  452
Rotator seen nodes :  322
EdgeDirector seen nodes :  322
BuildOrder seen nodes :  322
DetectTriangles seen nodes :  312
EdgeDirector seen nodes :  287
FunctionQ seen nodes :  443
GraphTrim seen node

MakeInstructions seen nodes :  277
SpatialRoot seen nodes :  120
EdgeDirector seen nodes :  120
BuildOrder seen nodes :  120
KDTreeIndex seen nodes :  120
Rotator seen nodes :  80
EdgeDirector seen nodes :  80
BuildOrder seen nodes :  80
DetectTriangles seen nodes :  80
EdgeDirector seen nodes :  72
FunctionQ seen nodes :  120
GraphTrim seen nodes :  120
EdgeDirector seen nodes :  116
FunctionQ seen nodes :  116
DetectElbows seen nodes :  115
EdgeDirector seen nodes :  116
Scale seen nodes :  116
LabelConnector seen nodes :  116
FunctionQ seen nodes :  115
MakeInstructions seen nodes :  116
SpatialRoot seen nodes :  120
EdgeDirector seen nodes :  120
BuildOrder seen nodes :  120
KDTreeIndex seen nodes :  120
Rotator seen nodes :  80
EdgeDirector seen nodes :  80
BuildOrder seen nodes :  80
DetectTriangles seen nodes :  80
EdgeDirector seen nodes :  72
FunctionQ seen nodes :  120
GraphTrim seen nodes :  120
EdgeDirector seen nodes :  116
FunctionQ seen nodes :  116
DetectElbows seen nod

MakeInstructions seen nodes :  45
SpatialRoot seen nodes :  12
EdgeDirector seen nodes :  12
BuildOrder seen nodes :  12
KDTreeIndex seen nodes :  12
Rotator seen nodes :  7
EdgeDirector seen nodes :  7
BuildOrder seen nodes :  7
DetectTriangles seen nodes :  7
EdgeDirector seen nodes :  7
FunctionQ seen nodes :  13
GraphTrim seen nodes :  13
EdgeDirector seen nodes :  13
FunctionQ seen nodes :  13
DetectElbows seen nodes :  12
EdgeDirector seen nodes :  13
Scale seen nodes :  13
LabelConnector seen nodes :  13
FunctionQ seen nodes :  12
MakeInstructions seen nodes :  13
SpatialRoot seen nodes :  60
EdgeDirector seen nodes :  60
BuildOrder seen nodes :  60
KDTreeIndex seen nodes :  60
Rotator seen nodes :  40
EdgeDirector seen nodes :  40
BuildOrder seen nodes :  40
DetectTriangles seen nodes :  40
EdgeDirector seen nodes :  36
FunctionQ seen nodes :  60
GraphTrim seen nodes :  60
EdgeDirector seen nodes :  60
FunctionQ seen nodes :  60
DetectElbows seen nodes :  59
EdgeDirector seen n

MakeInstructions seen nodes :  9
SpatialRoot seen nodes :  12
EdgeDirector seen nodes :  12
BuildOrder seen nodes :  12
KDTreeIndex seen nodes :  12
Rotator seen nodes :  12
EdgeDirector seen nodes :  12
BuildOrder seen nodes :  12
DetectTriangles seen nodes :  12
EdgeDirector seen nodes :  12
FunctionQ seen nodes :  12
GraphTrim seen nodes :  12
EdgeDirector seen nodes :  12
FunctionQ seen nodes :  7
DetectElbows seen nodes :  6
EdgeDirector seen nodes :  7
Scale seen nodes :  7
LabelConnector seen nodes :  7
FunctionQ seen nodes :  6
MakeInstructions seen nodes :  7
SpatialRoot seen nodes :  12
EdgeDirector seen nodes :  12
BuildOrder seen nodes :  12
KDTreeIndex seen nodes :  12
Rotator seen nodes :  7
EdgeDirector seen nodes :  7
BuildOrder seen nodes :  7
DetectTriangles seen nodes :  7
EdgeDirector seen nodes :  7
FunctionQ seen nodes :  13
GraphTrim seen nodes :  13
EdgeDirector seen nodes :  13
FunctionQ seen nodes :  13
DetectElbows seen nodes :  12
EdgeDirector seen nodes :  

In [141]:
RL(sks)
e = gutils.edge_with_id(rn0, 7676757)
ex1, ex2 = e.target.successors(edges=True)

print(e.target.tmps)
n1 = Node(e.source.geom)
n2 = Node(e.target.geom, **e.target.tmps)

n3 = Node(ex1.target.geom)
n4 = Node(ex2.target.geom)

e1 = n1.connect_to(n2, **e.tmps)
e2 = n2.connect_to(n3, **ex1.tmps)
e3 = n2.connect_to(n4, **ex2.tmps)

e2 = sks.align_vertical(e2)
e3 = sks.align_vertical(e3)
print(e1.tmps)
print(e2.tmps)
print(e3.tmps)
for n in [n1, n2, n3, n4]:
    n.geom = gutils.tuplify(n.as_np + 5)
    

res = sks.MakeInstructions()(n1, shrink=-0.05, use_radius=True)
save_instr(res)

{'pt_index': 3332, 'order': 10, 'is_tee': True}
{'is_pipe': True, 'radius': 0.0688976377952756, 'order': 9}
{'is_pipe': True, 'radius': 0.0374015748031496, 'order': 14, 'tap_edge': True}
{'is_pipe': True, 'radius': 0.06889763779527561, 'order': 17}
929428 8201580 3773741
MakeInstructions seen nodes :  4
creating insturcionts 4


[[1,
  929428,
  257.68469898330824,
  4.999962020543802,
  4.971847746250392,
  252.8500183560248,
  4.999962185555826,
  4.9259549739175466,
  0.16666666666666666],
 [1,
  3773741,
  252.80002060851893,
  4.999962187262295,
  4.9754803747826015,
  252.80002060851893,
  4.999962187262295,
  5.342748818168488,
  0.16666666666666666],
 [1,
  8201580,
  252.75002286112675,
  4.999962188806639,
  4.925005763670231,
  241.76991750883252,
  4.999962527963154,
  4.820775467566609,
  0.16666666666666666],
 [5, 3401756, 929428, 1, 8201580, 0, 3773741, 0]]

In [454]:
import lib.meshcat.src.meshcat.transformations as txf


def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

def eulerAnglesToRotationMatrix(theta) :   
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])       
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])      
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])    
    R = np.dot(R_z, np.dot( R_y, R_x ))
    return R


def rotationMatrixToEulerAngles(R) :
    # Calculates rotation matrix to euler angles
    # The result is the same as MATLAB except the order
    # of the euler angles ( x and z are swapped ).
    assert(isRotationMatrix(R))
     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([z, y, x])


def angl(u, v):
    return np.arccos( np.dot( u, v) )


def euler_composition(m1, m2, m3):
    return np.dot(m3, np.dot(m2, m1))


def ngc(u, v):
    x_angle = angl(u[[1,2]], v[[1,2]])
    y_angle = angl(u[[0,2]], v[[0,2]] ) 
    z_angle = angl(u[[0,1]], v[[0,1]] )    
    return np.array([x_angle, y_angle, z_angle])

In [521]:
rn0.successors()
import transforms3d.euler as euler
import transforms3d.taitbryan as tb



2

In [636]:
RL(revit)
RL(sks)

bld = sks.MakeInstructionsNodeBased()
bld.on_default(rn0)
print(bld._res[-2])

test3 = [rn0.successors()[1],  rn0,  rn0.successors()[0]]
test4 = [rn0, 
         ds2 , 
         ds2.successors()[1] ]


bld = sks.MakeInstructionsNodeBased()
bld.on_default(ds2)
print(bld._res[-2])
revit.Command.place_family_point_angle(*test4, family="Tee - Generic")

[12, 'Tee - Generic', 6906700, 2, 526.848367108863, -51.715957739736886, 1.6515011751184112, -0.5773502693683984, 0.5773502691002396, -0.5773502691002395, 2.0943951021250364, 0.9999916691351617, 4.6446102012275806e-10, -0.0, -0.0]
[12, 'Tee - Generic', 2650973, 2, 526.8483671045805, -42.49549076014023, 1.6138641167567014, -0.5773502690108531, -0.577350269279012, 0.577350269279012, 2.0943951026613545, -0.9999916691351617, -4.6446102012275806e-10, 0.0, -0.0]


['Tee - Generic',
 2650973,
 2,
 526.8483671045805,
 -42.49549076014023,
 1.6138641167567014,
 -0.5773502690108531,
 -0.577350269279012,
 0.577350269279012,
 2.0943951026613545,
 -0.9999916691351617,
 -4.6446102012275806e-10,
 0.0,
 -0.0]

In [653]:
RL(revit)
RL(sks)
res = sks.MakeInstructionsNodeBased()(rn0, shrink=-0.05, 
                                      use_radius=True)
_ = save_instr(res)
_

MakeInstructionsNodeBased seen nodes :  290
creating insturcionts 833


[[1, 7277417, 6906700, 2650973, 0.03740157480314961],
 [1,
  9251678,
  6906700,
  526.848367108863,
  -51.715957739736886,
  1.859877920273652,
  0.08333333333333333],
 [12,
  'Tee - Generic',
  6906700,
  2,
  526.848367108863,
  -51.715957739736886,
  1.6515011751184112,
  -0.5773502693683984,
  0.5773502691002396,
  -0.5773502691002395,
  2.0943951021250364,
  0.9999916691351617,
  4.6446102012275806e-10,
  -0.0,
  -0.0,
  1.5667144457565891],
 [1, 5174695, 2650973, 909403, 0.03740157480314961],
 [1,
  8899121,
  2650973,
  526.8483671045805,
  -42.49549076014023,
  1.822240861911942,
  0.08333333333333333],
 [12,
  'Tee - Generic',
  2650973,
  2,
  526.8483671045805,
  -42.49549076014023,
  1.6138641167567014,
  -0.5773502690108531,
  -0.577350269279012,
  0.577350269279012,
  2.0943951026613545,
  -0.9999916691351617,
  -4.6446102012275806e-10,
  0.0,
  -0.0,
  1.5667144457565891],
 [1, 3423236, 909403, 8481463, 0.0374015748031496],
 [1,
  6111873,
  909403,
  526.8483670998708,

In [677]:
from shapely.ops import linemerge, nearest_points

# eg1 = gutils.edge_with_id(rn0, 9835536)
#eg2 = gutils.edge_with_id(rn0, 1512714)
eg1 = gutils.edge_with_id(rn0, 1512714)
eg2 = gutils.edge_with_id(rn0, 3172226)
3172226
l1, l2 = eg1.curve.extend(10, 10), eg2.curve.extend(10, 10)
print(np.round(l1.numpy))
print(np.round(l2.numpy))
p1, p2 = nearest_points(l1, l2)
print(list(p1.coords), p2.coords)
pp1 = l2.line.points[0].projected_on(l1.line)
pp2 = l1.line.points[1].projected_on(l2.line)
print(np.round(eg1.curve.numpy))
print(np.round(eg2.curve.numpy))
pp1.distance_to(pp2), 1/12

[[527.   0.  11.]
 [527.   0. -10.]]
[[537.  -0.   0.]
 [506.  -0.  -0.]]
[(526.9735870942637, 0.035935875816308666)] <shapely.coords.CoordinateSequence object at 0x7f8d886a5198>
[[527.   0.   1.]
 [527.   0.   0.]]
[[527.  -0.  -0.]
 [516.  -0.  -0.]]


(0.07651440555615209, 0.08333333333333333)

In [36]:
eg1 = gutils.edge_with_id(rn0, 7515578)

eg1.tmps

{'is_pipe': False, 'order': 164, 'is_elbow': True}

In [231]:
def copy_translate(node, t=np.array([0, 0, 10])):
    xf = lambda x, kw: Node(gutils.tuplify(x.as_np + t), **kw)
    nd = xf(node, node.tmps)
    for e, n in node.successors(both=True):
        new_nd = xf(n, {})
        nd.connect_to(new_nd, **e.tmps)
    for e, n in node.predecessors(both=True):
        new_nd = xf(n, {})
        new_nd.connect_to(nd, **e.tmps)
    return nd

In [232]:
# tap = gutils.edge_with_id(rn0, 5851648)
inn = gutils.edge_with_id(rn0, 1407181)
# out = gutils.edge_with_id(rn0, 204995)
tee = copy_translate(inn.target)
print(tee.tmps)
inn2 = tee.predecessors()[0]

res2 = sks.MakeInstructions()(inn2, shrink=-0.05, use_radius=True)
save_instr(res2)

{'pt_index': 3057, 'order': 72, 'is_tee': True}
MakeInstructions seen nodes :  4
creating insturcionts 4


[[1,
  8181741,
  468.19801874219564,
  -55.34417201144305,
  11.63056683102958,
  468.1979815857606,
  -95.09646043286497,
  11.47420610239293,
  0.16666666666666666],
 [1,
  6378434,
  468.1979814891446,
  -95.19645654891363,
  11.474600793440707,
  468.19796298409227,
  -113.74421473890237,
  11.693983577314778,
  0.16666666666666666],
 [1,
  2961643,
  468.24798020479204,
  -95.14648085647212,
  11.473644759134382,
  491.41471935124014,
  -95.15612329084985,
  11.304673202500316,
  0.16666666666666666],
 [5, 675221, 8181741, 1, 6378434, 0, 2961643, 0]]

In [224]:
RL(sks)
RL(gp)
RL(gutils)
def prslope(node, **kwargs):
    tol = 128
    for x in node.successors(edges=True):
        print(round(12 *gutils.slope(x) * tol) / tol )
    return node
        
rn0.successors(edges=True)[0].curve.line.numpy
prslope(rn0)
# gp.FunctionQ(prslope)(rn0)

0.046875


In [203]:
import inspect
RL(src.propogate.propagators)
RL(src.propogate)
RL(gp)
inspect.getmro(gp.GraphTrim)

(src.propogate.propagators.GraphTrim,
 src.propogate.base.QueuePropogator,
 object)

## TESTING SERVER

In [77]:
# TEST TEES
RL(sks)

RN = Node((270, 0, 0))
n2 = Node((270, 5, 0), head_tee=True)

n3 = Node((270, 10, 0))
nu = Node((270, 5, 1))

RN.connect_to(n2, is_pipe=True)
n2.connect_to(n3, is_pipe=True)
n2.connect_to(nu, remove_head=True)


r2 = sks.MakeInstructions()(RN, shrink=-0.05, use_radius=True)
r2 = save_instr(r2)
r2

MakeInstructions seen nodes :  4


[[1, 9175618, 270.0, 0.05, 0.0, 270.0, 4.95, 0.0, 0.16666666666666666]]

In [51]:
# TEST COUOPLINGS
RL(sks)

RN = Node((275, 0, 0))
n1 = Node((275, 10, 0), is_coupling=True)
n2 = Node((275, 15, 0))

RN.connect_to(n1, is_pipe=True)
n1.connect_to(n2, is_pipe=True)

r2 = sks.MakeInstructions()(RN, shrink=-0.05, use_radius=True)
r2 = save_instr(r2)
r2

MakeInstructions seen nodes :  3


[[1, 6284211, 275.0, 0.05, 0.0, 275.0, 9.95, 0.0, 0.16666666666666666],
 [1, 9694699, 275.0, 10.05, 0.0, 275.0, 15.0, 0.0, 0.16666666666666666],
 [8, 6284211, 1, 9694699, 0]]

In [62]:
# TEST ELBOWS
RL(sks)

RN = Node((280, 0, 0))
n1 = Node((280, 10, 0), is_elbow=True)
n2 = Node((285, 10, 0), is_elbow=True)
n3 = Node((285, 15, 0))

RN.connect_to(n1, is_pipe=True)
n1.connect_to(n2, is_pipe=True)
n2.connect_to(n3, is_pipe=True)

r2 = sks.MakeInstructions()(RN, shrink=-0.1, use_radius=True)
r2 = save_instr(r2)
r2

MakeInstructions seen nodes :  4


[[1, 45569, 280.0, 0.1, 0.0, 280.0, 9.9, 0.0, 0.16666666666666666],
 [1, 7666367, 280.1, 10.0, 0.0, 284.9, 10.0, 0.0, 0.16666666666666666],
 [1, 3906340, 285.0, 10.1, 0.0, 285.0, 15.0, 0.0, 0.16666666666666666],
 [4, 45569, 1, 7666367, 0],
 [4, 7666367, 1, 3906340, 0]]

In [83]:
RN = Node((280, 0, 0))
r2 = Node((280, 1, 10))
egd = RN.connect_to(r2, is_pipe=True)

RL(sks)
RL(gutils)
sks.is_vertical(egd)

0.004962809790010847

----------------------------------------------

In [1493]:
# todo
def low_line(pts_index):
    lowest_ix = np.argsort(pts_index[:, -1])
    lowest_spr = pts_index[lowest_ix[0:2]]
    return Line(Point(lowest_spr[0]), Point(lowest_spr[1]))


CAD_Line = low_line(pts_ind)
p1, p2 = CAD_Line.points
rot = sks.Rotator(CAD_ANGLE)
cln = CAD_Line.moved(rot.M)


REV_Line = low_line(rvpts_index)

print(CAD_Line.length / REV_Line.length)

viz3.viz(REV_Line, handle='low/rev', thickness=1)
viz3.viz(cln, handle='low/cad', thickness=1)



0.4724310517187339


In [177]:
import src.render.render_propogators as rps
import src.process as process
import json
RL(rps)
RL(sks)
RL(process)
SHRINK = -0.05
OUTPATH = '/home/psavine/source/viper/data/out/sks/'

def save_outs(args, pth):
    geom, inds, syms = args
    res = {'geom': geom, 'indicies': inds, 'symbols': syms}
    print(len(geom), len(inds), len(syms))
    st = str(round(time.time(), 0))
    with open('{}to-revit-{}.json'.format(OUTPATH, pth), 'w') as F:
        json.dump(res, F)
        
        
def _prepare(start, end, last=False):
    Shrink = -0.01
    edge = start.edge_to(end)
    radius = edge.get('radius')
    if radius is None:
        radius = 1.
    crv = geom.MepCurve2d(start.geom, end.geom)
    if last is False:
        p1, p2 = crv.extend(Shrink, Shrink).points
    else:
        p1, p2 = crv.extend(Shrink, 0).points

    if math.isnan(p1[0]):
        print(start, end, p1, p2)
    vec = list(p1) + list(p2) + [radius]
    return vec    


def _preparep(start, end, last=False):
    Shrink = -0.01
    edge = start.edge_to(end)
    radius = edge.get('radius')
    if radius is None:
        radius = 1.
    vec = list(start.geom) + list(end.geom) + [radius]
    return vec    


def TO_REVIT(root_nd, n, prep=_preparep):
    anno = sks.Annotator('$create', mapping={'has_head': 3})
    gp.EdgeDirector()(root_nd)
    anno(root_nd)

    gp.BuildOrder()(root_nd)

    print(anno._pos)
    res = process.finalize(root_nd, prep)

    save_outs(res, n)

def TO_REVIT2(root_nd, n):
    anno = sks.Annotator('$create', mapping={'has_head': 3})
    gp.EdgeDirector()(root_nd)
    anno(root_nd)

    gp.BuildOrder()(root_nd)

    print(anno._pos)
    res = process.finalize(root_nd, _prepare)

    save_outs(res, n)

EdgeDirector seen nodes :  292
Annotator seen nodes :  292
BuildOrder seen nodes :  292
0
291 180 0


In [153]:
RL(geombase)
pts = []
for msh in train_mesh:
    if msh.category =='Pipes':
        ln = geombase.as_cylinder(msh)
        if ln is not None:
            pts.append(ln.numpy)
rvpts_index = np.concatenate(pts)

In [178]:
import src.propogate.propagators as gpp
RL(gpp)

def buildo(node, value=-2, **kwargs):
    return node.get('order', -1) == value

def slope(crv):
    npp = crv.numpy
    # print(crv.length)
    z = npp[0, -1] - npp[1, -1]
    
    return np.abs(z / Line(*map(Point, crv.points)).length)



fedge = rn1.successors(edges=True)[0]

n10 = gpp.Where(buildo, edges=True)(fedge, value=10)
n11 = gpp.Where(buildo, edges=True)(fedge, value=11)
n8 = gpp.Where(buildo, edges=True)(fedge, value=8)

print(n8.similar_direction(n11))
print(n8.similar_direction(n10))
print(n10.similar_direction(n8))
# print(n10.direction(n8))

edges__ = [n10, n11, n8]

np.allclose(n10.direction, n10.direction)

[print(np.round(x.direction, 3)) for x in edges__]
preped = [_prepare(x.source, x.target) for x in edges__]
res = [preped[0][0:3], preped[1][0:3], preped[2][3:6]]
for i, r in enumerate(res):
    V.viz_point(r, handle='test/' + str(i), radius=0.04)

False
False
False
[-0.001  1.     0.013]
[0. 0. 1.]
[ 0.     1.    -0.008]


In [154]:

viz3.viz_point(rn1.geom, handle='test')

In [169]:
scs = rn1.successors(edges=True)
[x.curve.length for x in scs]
scs[1].target.id
Line(*map(Point, scs[1].curve.points)).length


0.25

In [152]:
x = [[print(x.curve.length) for x in n.successors(edges=True)] for n in rn1.__iter__()]

4.424903815802851
0.0
4.724409448892871
0.0
4.724416227465029
0.0
4.724402670148194
0.0
4.4881889763999965
0.0
1.4957284255232712
0.0
2.478527986093234
5.669292060701303
1.4957555886543963
4.682671853122132
4.488188976346123
0.0
4.724395891408776
0.0
4.724423006181383
0.0
4.72440944888227
0.0
4.424903412963972
0.0
0.067175219912541
0.9865987346366425
0.07236078501142551
2.1760124066530855
0.034878687351237996
26.190306445110405
4.717697120001906
0.9564337814230855
5.669293752049265
1.4958002112352013
1.4957740893192808
2.478518031069224
4.488188976443702
0.0
4.7244094488386565
0.0
4.724409448839083
0.0
4.724409448774793
0.0
4.424903789235771
0.0
4.488188976393822
0.0
4.724409448746627
0.0
4.724409448799719
0.0
4.724409448872876
0.0
4.42490342864997
0.0
5.669296178601118
1.4958606619651829
0.9363991779603119
1.4958965948616998
0.23363305708584128
0.37765748035644303
0.18956692907133119
0.37765748029841023
4.488188976393715
0.0
4.724409448788109
0.0
4.724409448798654
0.0
4.72440944882455

# --------------------------------------------------------------

In [1385]:
for i in range(len(comps_starts)):
    viz3.viz_iter(root_nodes[i], handle='nodec/{}'.format(i),
                  clear=True, opacity=0.3)

DetectTriangles seen nodes :  330   
DetectElbows seen nodes :  347   
elbows: 28 , tirangles 18   


In [1414]:
RL(sks)

RL(geo)
from lib.geo import Line, Point # , Movement
import lib.meshcat.src.meshcat.transformations as txf


def rotator(orienting_edge):
    rnt = np.array([orienting_edge.source.geom, 
                    orienting_edge.target.geom])

    rnt[:, -1] = 0

    norm = Line(Point(rnt[1]), Point(rnt[0]))
    print(norm)
    base = Line(Point(0., 0., 0.), Point(1., 0, 0))
    m = geo.Movement(norm, base)
    
    angle = norm.angle_to(base)
    print(angle)
    return m
    # return -txf.rotation_matrix(angle, [0.,0., 1], rnt[1])[:3, :3]
    

edg = edge_with_id(ROOT, 4438916)
arr = np.asarray([list(edg.source.geom), list(edg.target.geom)])
print(np.round(arr, 3))
print(edg.curve.length)
M = rotator(edg)

print(M.q, M.dr)
p1 = Point(edg.source.geom)
p2 = Point(edg.target.geom)
ln = Line(Point(edg.source.geom), Point(edg.target.geom))


md = ln.moved(M)
print(ln.length, md.length)



[[ 2.42334e+02  0.00000e+00 -1.26000e-01]
 [ 2.36876e+02  0.00000e+00 -1.78000e-01]]
5.4582469151953035
Line(Point(236.876, 0, 0), Point(242.334, 0, 0))
0.0
None [0. 0. 0.]
5.458492828173617 5.458492828173617


### Connect Heads

## MISC TESTING


In [1031]:
import src.propogate
# RL(src.propogate.base)
RL(sks)

root = edge_with_id(root_nodes[0], 1275321)
nd = root.source
print(root)
sks.EdgeDirectorQ()(nd)


<Edge>:((58.17832921906061, -128.39112719713268, 0.3211833299301574), (57.19465048862049, -124.14188319947925, 0.29480801374617727)), {'is_pipe': True}


In [1035]:
VIZ['nodec'].delete() 
viz3.viz_iter(nd, handle='nodec/0', 
                   opacity=0.3)  

len(list(nd.__iter__(fwd=True, bkwd=True)))

1158

In [357]:
sprinks_arr = sprinklers3
sprinks_2d = sprinks_arr[:, 0:2]
sprinks_tree = kdtree.KDTree(sprinks_arr[:, 0:2])

pipe_2d = np.concatenate(plines)[:, 0:2]
pipe_2dtree = kdtree.KDTree( pipe_2d)
tr = sprinks_tree.query_ball_tree(pipe_2dtree, 1)

sprinks_to_pipes = dict()
pipes_to_sprinks = defaultdict(set)
sprinks_to_pipes_d = dict()
for i, x in enumerate(tr):
    if len(x) > 0:
        dists, pipe_ixs = sprinks_tree.query(pipe_2d[x])
        sprinks_to_pipes[i] = x[np.argmin(dists)]
        pipes_to_sprinks[x[np.argmin(dists)]].add(i) 
        sprinks_to_pipes_d[i] = dists[np.argmin(dists)]


array([  391.57757842, -7925.38562256,  1139.87186712])

## TRY WITH WHOLE SAT

In [96]:
headers, data = lines[0:3], lines[3:]

In [110]:
def dec_sat(bin_line):
    return bin_line.decode('ascii').split(' ')


def recur_bod(ix):
    
    seen = set()
    others = []
    q = [ix]
    while q:
        this_ix = q.pop()
        # print(this_ix)
        if this_ix in seen:
            continue
            
        seen.add(this_ix)
        raw = data[this_ix]
        
        chunks = dec_sat(raw)
        others.append(chunks)
        print(this_ix, chunks)
        for c in chunks:
            if c[0] == '$' and c != '$-1':
                todo = int(c[1:])
                # print('todo', todo)
                q.append(todo) 

    return others
            

# data[108302: 108306]
recur_bod(1)

1 ['body', '$-1', '-1', '$-1', '$21463', '$-1', '$21464', '#\r\n']
21464 ['transform', '$-1', '-1', '1', '0', '0', '0', '1', '0', '0', '0', '1', '-1101197.235013285652', '109246.670983092743', '832.9999939502620236', '1', 'rotate', 'no_reflect', 'no_shear', '#\r\n']
21463 ['lump', '$-1', '-1', '$-1', '$-1', '$64713', '$1', '#\r\n']
64713 ['shell', '$-1', '-1', '$-1', '$-1', '$-1', '$86505', '$-1', '$21463', '#\r\n']
86505 ['face', '$108301', '-1', '$-1', '$-1', '$108302', '$64713', '$-1', '$108303', 'reversed', 'double', 'out', '#\r\n']
108303 ['\t0.1691308163470010006', '2', '0.171370211494747321', '3', '\r\n']
108302 ['\t0.1624126309037620397', '3', '0.1668914211992546803', '2', '0.1680111187731278544', '2', '0.1685709675600644275', '2', '0.1688508919535327002', '2', '\r\n']
108301 ['\t0', '2', '1', '2', '\r\n']


[['body', '$-1', '-1', '$-1', '$21463', '$-1', '$21464', '#\r\n'],
 ['transform',
  '$-1',
  '-1',
  '1',
  '0',
  '0',
  '0',
  '1',
  '0',
  '0',
  '0',
  '1',
  '-1101197.235013285652',
  '109246.670983092743',
  '832.9999939502620236',
  '1',
  'rotate',
  'no_reflect',
  'no_shear',
  '#\r\n'],
 ['lump', '$-1', '-1', '$-1', '$-1', '$64713', '$1', '#\r\n'],
 ['shell',
  '$-1',
  '-1',
  '$-1',
  '$-1',
  '$-1',
  '$86505',
  '$-1',
  '$21463',
  '#\r\n'],
 ['face',
  '$108301',
  '-1',
  '$-1',
  '$-1',
  '$108302',
  '$64713',
  '$-1',
  '$108303',
  'reversed',
  'double',
  'out',
  '#\r\n'],
 ['\t0.1691308163470010006', '2', '0.171370211494747321', '3', '\r\n'],
 ['\t0.1624126309037620397',
  '3',
  '0.1668914211992546803',
  '2',
  '0.1680111187731278544',
  '2',
  '0.1685709675600644275',
  '2',
  '0.1688508919535327002',
  '2',
  '\r\n'],
 ['\t0', '2', '1', '2', '\r\n']]